In [1]:
import os

In [15]:
%pwd

'd:\\college_work\\4th year\\Sem7th\\Project\\OBE\\Paper- Blooms taxanomy\\Boolm-s-Level-Detection-A-MLOPS-Project'

In [3]:
os.chdir("../")
%pwd

'd:\\college_work\\4th year\\Sem7th\\Project\\OBE\\Paper- Blooms taxanomy\\Boolm-s-Level-Detection-A-MLOPS-Project'

## Data Cleaning

In [16]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib.inline

UsageError: Line magic function `%matplotlib.inline` not found.


In [ ]:
df = pd.read_csv("./artifacts/data_ingestion/blooms_dataset.csv")
print(df.info())

df.head(10)

In [ ]:
df["Level"].value_counts().plot(kind="bar")

In [29]:
# print(f" {df[df['Text'].str.startswith(' ')].shape}")
# print(f" {df[df['Text'].str.endswith(' ')].shape}")

# df['Text'] = df['Text'].str.strip()

# print(f" {df[df['Text'].str.startswith(' ')].shape}")
# print(f" {df[df['Text'].str.endswith(' ')].shape}")

Creating another dataframe with lower cases 
because uncased data is only used with pretrained model like BERD because they are case sensetive 

In [110]:
bert_df = df
print(f"is lower {df[df['Text'].str.islower()].shape}")
df["Text"] = df["Text"].str.lower()
print(f"is lower {df[df['Text'].str.islower()].shape}")
# removing some extra space
df["Text"] = df["Text"].str.strip()


is lower (600, 2)
is lower (600, 2)


### Data Augmentation

In [ ]:
"""
augmented each Text input 3 times with following parameter with nlpaug library

```ContextualWordEmbsAug(model_path="bert-base-uncased", action="insert")```

before augmendatation data shape : 600,2
total 6 unique calsses 100 each 

augmented data shape : 1800,2
total 6 unique classes 300 each
"""

import pandas as pd
import nlpaug.augmenter.word as naw
from tqdm import tqdm


def augment_text(text):
    aug = naw.ContextualWordEmbsAug(model_path="bert-base-uncased", action="insert")
    augmented_text = aug.augment(text)
    return augmented_text


def augment_rows_pytorch(row):
    augmented_texts = [augment_text(row["Text"]) for _ in range(3)]
    text_list = [text for text in augmented_texts]
    level_list = [row["Level"]] * 3
    return {"Text": text_list, "Level": level_list}


augmented_data = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Augmenting Text"):
    augmented_data.append(augment_rows_pytorch(row))

augmented_data_flat = [
    {"Text": Text, "Level": Level}
    for row in augmented_data
    for Text, Level in zip(row["Text"], row["Level"])
]

augmented_df = pd.DataFrame(augmented_data_flat)

print("Original DataFrame:")
print(df.head())

print("\nAugmented DataFrame:")
print(augmented_df.head())

print("\nFinal DataFrame Shape:", augmented_df.shape)


augmented_df.to_csv("./artifacts/data_transformation/augmented_bloom_data.csv")

In [19]:
# loading the augmented data 
df = pd.read_csv('./artifacts/data_transformation/augmented_bloom_data.csv',index_col=0)
print(df.info())
df.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 1800 entries, 0 to 1799
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    1800 non-null   object
 1   Level   1800 non-null   object
dtypes: object(2)
memory usage: 42.2+ KB
None


,Text,Level
0,['is about average what proportion part of in ...,Remember
1,['about seeing what huge proportion of society...,Remember
2,['how about what proportion of that the popula...,Remember
3,['correctly label the three brain lobes are in...,Remember
4,['most correctly correct label the brain lobes...,Remember
5,['correctly label the two brain lobes indicate...,Remember
6,['and define compound by interest.'],Remember
7,['this define of compound interest.'],Remember
8,['and define their compound interest.'],Remember
9,['define those four types part of data traceab...,Remember


In [20]:
# Removing the square bracket of each text 
def remove_bracket(text):
    return text[2:-2]

text = "[/dlfjasldfjasldfjlsdfj]"
df['Text'] = df['Text'].apply(lambda x: remove_bracket(x))
df.head(10)

,Text,Level
0,is about average what proportion part of in th...,Remember
1,about seeing what huge proportion of society t...,Remember
2,how about what proportion of that the populati...,Remember
3,correctly label the three brain lobes are indi...,Remember
4,most correctly correct label the brain lobes i...,Remember
5,correctly label the two brain lobes indicated ...,Remember
6,and define compound by interest.,Remember
7,this define of compound interest.,Remember
8,and define their compound interest.,Remember
9,define those four types part of data traceabil...,Remember


### Tokenization
Performing the nltk and berttokenization to show the difference

In [21]:
df = df.drop(df[df['Text'].str.contains(r'\.{2,}',regex=True)].index)
df.shape

(1622, 2)

In [9]:
# from transformers import BertTokenizer

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# test = "this is an example of ANN"

# tokens = tokenizer(test,return_tensors='pt')
# print(tokens)

In [22]:
from nltk.tokenize import word_tokenize

df["Tokens"] = [word_tokenize(text) for text in df["Text"]]
df['Tokens'].head(10)

0    [is, about, average, what, proportion, part, o...
1    [about, seeing, what, huge, proportion, of, so...
2    [how, about, what, proportion, of, that, the, ...
3    [correctly, label, the, three, brain, lobes, a...
4    [most, correctly, correct, label, the, brain, ...
5    [correctly, label, the, two, brain, lobes, ind...
6             [and, define, compound, by, interest, .]
7            [this, define, of, compound, interest, .]
8          [and, define, their, compound, interest, .]
9    [define, those, four, types, part, of, data, t...
Name: Tokens, dtype: object

In [147]:
df.to_csv('./artifacts/data_transformation/clean_blooms_dataset.csv',index=False)

## MLops steps

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    clean_data_path: Path
    augmented_data_path: Path
    new_data_path: Path

In [9]:
from blooms.constants import *
from blooms.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_tranformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            clean_data_path=config.clean_data_path,
            augmented_data_path=config.augmented_data_path,
            new_data_path=config.new_data_path,
        )
        return data_transformation_config

In [53]:
from sklearn.model_selection import train_test_split
from blooms import logger
import pandas as pd
from nltk.tokenize import word_tokenize
import pandas as pd
import nlpaug.augmenter.word as naw
from tqdm import tqdm


class DataTrnsformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def data_preprocessing(self):
        df = pd.read_csv(self.config.data_path, index_col=None)
        df["Text"] = df["Text"].str.strip()
        df["Text"] = df["Text"].str.lower()
        df.to_csv(self.config.clean_data_path, index=False)

    def augment_text(text):
        """
        augmented each Text input 3 times with following parameter with nlpaug library

        ```ContextualWordEmbsAug(model_path="bert-base-uncased", action="insert")```

        before augmendatation data shape : 600,2
        total 6 unique calsses 100 each 

        augmented data shape : 1800,2
        total 6 unique classes 300 each
        """
        aug = naw.ContextualWordEmbsAug(model_path="bert-base-uncased", action="insert")
        augmented_text = aug.augment(text)
        return augmented_text

    def augment_rows(row):
        augmented_texts = [augment_text(row["Text"]) for _ in range(3)]
        text_list = [text for text in augmented_texts]
        level_list = [row["Level"]] * 3
        return {"Text": text_list, "Level": level_list}

    def data_augmentation(self):
        df = pd.read_csv(self.config.clean_data_path, index_col=None)
        augmented_data = []
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Augmenting Text"):
            augmented_data.append(augment_rows(row))
        augmented_data_flat = [
            {"Text": Text, "Level": Level}
            for row in augmented_data
            for Text, Level in zip(row["Text"], row["Level"])
        ]
        augmented_df = pd.DataFrame(augmented_data_flat)
        augmented_df.to_csv(self.config.augmented_data_path)
        logger.info(
            f"Done the augmentation! Saving the augmented data file at: {self.config.augmented_data_path}"
        )

    def tokenization(self):
        df = pd.read_csv(self.config.augmented_data_path, index_col=0)
        df['Text'] = df['Text'].apply(lambda x: x[2:-2])
        df = df.drop(df[df['Text'].str.contains(r'\.{2,}',regex=True)].index)
        df["Tokens"] = [word_tokenize(text) for text in df["Text"]]
        df = df.sample(frac=1, random_state=1).reset_index(drop=True)
        
        df.to_csv(self.config.new_data_path, index=False)
        logger.info(
            f"Done the tokenization! Saving the final cleaned file at: {self.config.new_data_path}"
        )

    def train_test_splitting(self):
        data = pd.read_csv(self.config.new_data_path)
        train, test = train_test_split(data)
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)
        logger.info(
            f"Done the Splitting! Saving the train and test  file at: {self.config.root_dir}"
        )
        logger.info(f"Splited data into training and testing sets")
        logger.info(train.shape)
        logger.info(test.shape)

In [55]:
try:
    config = ConfigurationManager()
    data_transfermation_config = config.get_data_tranformation_config()
    data_transfermation = DataTrnsformation(config=data_transfermation_config)
    data_transfermation.data_preprocessing()
    # data_transfermation.data_augmentation()
    data_transfermation.tokenization()
    data_transfermation.train_test_splitting()
except Exception as e:
    raise e

[2023-12-18 21:39:06: INFO: common: yaml file: config\config.yaml loaded succcessfully]
[2023-12-18 21:39:06: INFO: common: yaml file: params.yaml loaded succcessfully]
[2023-12-18 21:39:06: INFO: common: yaml file: schema.yaml loaded succcessfully]
[2023-12-18 21:39:06: INFO: common: created directory at : artifacts]
[2023-12-18 21:39:06: INFO: common: created directory at : artifacts/data_transformation]
[2023-12-18 21:39:06: INFO: 3519884042: Done the tokenization! Saving the final cleaned file at: artifacts/data_transformation/new_blooms_dataset.csv]
[2023-12-18 21:39:06: INFO: 3519884042: Done the Splitting! Saving the train and test  file at: artifacts/data_transformation]
[2023-12-18 21:39:06: INFO: 3519884042: Splited data into training and testing sets]
[2023-12-18 21:39:06: INFO: 3519884042: (1216, 3)]
[2023-12-18 21:39:06: INFO: 3519884042: (406, 3)]
